In [10]:
import sys, os, json
import re, nltk
from nltk import pos_tag
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn, gensim
from sklearn.decomposition import PCA
import itertools
from gensim.corpora import Dictionary
from collections import defaultdict
sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
os.chdir(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/data/')
from utils import preprocess_for_bow, experiment_result
from models.lda import LDAwrappers
from models.hdp import HDPwrapper
from models.gsdmm import MovieGroupProcessWrapper
from models.lftm import LFTMwrapper

In [11]:
with open('experiments_dataset1.json', 'r') as fout:
    res=json.load(fout)

In [58]:
with open('experiments_dataset2.json', 'r') as fout:
    res2=json.load(fout)

In [27]:
import pandas as pd
def to_df(res):
    params=defaultdict(list)
    for exp in res.keys():
        params['c_we'].append(res[exp]['coherence_metrics']['c_we']['c_we'])
        params['c_we_std'].append(res[exp]['coherence_metrics']['c_we']['c_we_std'])
        #params['num_topics'].append(res[exp]['number_topics'])
        for p in res[exp]['hyperparameters'].keys():
            params[p].append(res[exp]['hyperparameters'][p])
    #if params['num_topics'][0]==None:
    #    del params['num_topics']
    return pd.DataFrame(params)
    #return params


### LDA

#### DATASET 1

In [45]:
lda=to_df(res['lda_experiment'])
lda.iloc[lda.nlargest(5, 'c_we').index]

,c_we,c_we_std,num_topics,decay,passes
48,0.242069,0.087584,16,0.75,1
27,0.241629,0.049041,12,0.50,1
0,0.241507,0.046055,6,0.50,1
14,0.239710,0.087527,8,0.75,5
20,0.236841,0.057334,10,0.50,5


In [49]:
lda.iloc[lda.nsmallest(5, 'c_we_std').index]

,c_we,c_we_std,num_topics,decay,passes
6,0.213938,0.028108,6,0.90,1
18,0.199861,0.036153,10,0.50,1
50,0.216734,0.040622,16,0.75,5
8,0.208430,0.041338,6,0.90,5
39,0.205115,0.043111,14,0.75,1


In [36]:
import statsmodels.api as sm
y = lda['c_we']
x=lda.drop(columns=['c_we', 'c_we_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   c_we   R-squared:                       0.044
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.8945
Date:                Thu, 02 Nov 2023   Prob (F-statistic):              0.449
Time:                        09:01:50   Log-Likelihood:                 188.76
No. Observations:                  63   AIC:                            -369.5
Df Residuals:                      59   BIC:                            -361.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2229      0.009     25.301      0.0

In [37]:
y = lda['c_we_std']
x=lda.drop(columns=['c_we', 'c_we_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               c_we_std   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                 -0.015
Method:                 Least Squares   F-statistic:                    0.7010
Date:                Thu, 02 Nov 2023   Prob (F-statistic):              0.555
Time:                        09:02:35   Log-Likelihood:                 190.11
No. Observations:                  63   AIC:                            -372.2
Df Residuals:                      59   BIC:                            -363.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0561      0.009      6.502      0.0

#### DATASET 2


In [59]:
lda2=to_df(res2['lda_experiment'])
lda2.iloc[lda2.nlargest(5, 'c_we').index]

,c_we,c_we_std,num_topics,decay,passes
35,0.289717,0.109726,12,0.90,5
8,0.282400,0.152283,6,0.90,5
6,0.278735,0.126280,6,0.90,1
11,0.276158,0.139651,8,0.50,5
48,0.264954,0.146506,16,0.75,1


In [60]:
lda2.iloc[lda2.nsmallest(5, 'c_we_std').index]

,c_we,c_we_std,num_topics,decay,passes
44,0.239121,0.084175,14,0.90,5
16,0.224375,0.092427,8,0.90,2
22,0.225395,0.093985,10,0.75,2
14,0.257997,0.099312,8,0.75,5
24,0.259601,0.099661,10,0.90,1


In [61]:
y = lda2['c_we']
x=lda2.drop(columns=['c_we', 'c_we_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   c_we   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                 -0.027
Method:                 Least Squares   F-statistic:                    0.4474
Date:                Thu, 02 Nov 2023   Prob (F-statistic):              0.720
Time:                        10:55:27   Log-Likelihood:                 141.55
No. Observations:                  63   AIC:                            -275.1
Df Residuals:                      59   BIC:                            -266.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2358      0.019     12.646      0.0

In [62]:
y = lda2['c_we_std']
x=lda2.drop(columns=['c_we', 'c_we_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               c_we_std   R-squared:                       0.143
Model:                            OLS   Adj. R-squared:                  0.099
Method:                 Least Squares   F-statistic:                     3.268
Date:                Thu, 02 Nov 2023   Prob (F-statistic):             0.0274
Time:                        10:55:40   Log-Likelihood:                 179.37
No. Observations:                  63   AIC:                            -350.7
Df Residuals:                      59   BIC:                            -342.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1204      0.010     11.773      0.0

### GSDMM

#### DATASET 1

In [50]:
gsdmm=to_df(res['gsdmm_experiment'])
gsdmm.iloc[gsdmm.nlargest(5, 'c_we').index]

,c_we,c_we_std,K,alpha,beta,n_iters
1,0.365417,0.0,6,0.1,0.1,5
34,0.365417,0.0,12,0.2,0.2,5
2,0.352392,0.0,6,0.1,0.1,15
13,0.332914,0.0,9,0.1,0.1,5
61,0.322158,0.0,21,0.1,0.1,5


In [56]:
gsdmm.iloc[gsdmm.nsmallest(5, 'c_we').index]

,c_we,c_we_std,K,alpha,beta,n_iters
59,0.074076,0.0,18,0.2,0.2,15
53,0.087601,0.0,18,0.1,0.2,15
83,0.087601,0.0,24,0.2,0.2,15
8,0.125928,0.0,6,0.2,0.1,15
33,0.125928,0.0,12,0.2,0.2,1


In [54]:
gsdmm.iloc[gsdmm.nsmallest(5, 'c_we_std').index]

,c_we,c_we_std,K,alpha,beta,n_iters
0,0.184020,0.0,6,0.1,0.1,1
1,0.365417,0.0,6,0.1,0.1,5
2,0.352392,0.0,6,0.1,0.1,15
3,0.183047,0.0,6,0.1,0.2,1
4,0.240072,0.0,6,0.1,0.2,5


In [52]:
y = gsdmm['c_we']
x=gsdmm.drop(columns=['c_we', 'c_we_std'])
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   c_we   R-squared:                       0.103
Model:                            OLS   Adj. R-squared:                  0.058
Method:                 Least Squares   F-statistic:                     2.273
Date:                Thu, 02 Nov 2023   Prob (F-statistic):             0.0687
Time:                        09:10:45   Log-Likelihood:                 118.37
No. Observations:                  84   AIC:                            -226.7
Df Residuals:                      79   BIC:                            -214.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3047      0.034      8.869      0.0

### HDP